# Imports

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime

# Chrome Driver and Website

In [3]:
driver = webdriver.Chrome()

url_base = 'https://wdwpassport.com/wait-times/'
# park, ride index, final hour
parks = [['magic-kingdom', 24, 22], ['epcot', 12, 21], ['hollywood-studios', 12, 21], ['animal-kingdom', 10, 20]]

# Functions

## Check for closed parks

In [4]:
def park_closed():
    down_xpath = '//*[@id="app"]/div/div[1]/ul/li'
    check = driver.find_element_by_xpath(down_xpath)
    if 'Park Currently Closed' in check.text:
        return True
    return False

## Checks if the ride is down

In [5]:
def check_ride_down(n):
    try:
        down_xpath = '//*[@id="app"]/div/div/ul/li['+str(n)+']/div[1]/div[1]/div'
        driver.find_element_by_xpath(down_xpath)
    except NoSuchElementException:
        return False
    return True

## Finds the ride name and wait time

In [6]:
def ride(n):
    name_xpath = '//*[@id="app"]/div/div/ul/li['+str(n)+']/h3'
    time_xpath = '//*[@id="app"]/div/div/ul/li['+str(n)+']/div[1]/div/div'
    ride_name = driver.find_element_by_xpath(name_xpath)
    ride_time = driver.find_element_by_xpath(time_xpath)
    return ride_name.text, ride_time.text

## Collects the wait times 

In [11]:
def collect_times(url_base=url_base, parks=parks):
    wait_list = []
    for park in parks:
        if int(datetime.now().strftime("%H")) < park[2]:
            url = url_base + park[0]
            driver.get(url)
            time.sleep(1)
            if park_closed() == False:
                for n in range(2, park[1]):
                    if check_ride_down(n) == True:
                        a = ride(n)
                        wait_list+=[[date.today().strftime("%m/%d/%Y"),
                                     datetime.now().strftime("%H:%M"), 
                                     park[0], 
                                     a[0], 
                                     a[1]]]
    return wait_list

## Appends csv data

In [7]:
def update_file():
    times = collect_times()
    if len(times) > 0:
        df_new = pd.DataFrame(np.array(times), columns=['date', 'time', 'park', 'ride', 'wait'])
        df_old = pd.read_csv('wait_time.csv')
        df_combine = pd.concat([df_old, df_new])
        df_combine.to_csv(index=False, path_or_buf='wait_time.csv', mode='w+')

## Main

In [13]:
def main():
    while int(datetime.now().strftime("%H")) < 23:
        if int(datetime.now().strftime("%M")) % 5 == 0:
            update_file()
            time.sleep(295)

# Test

In [10]:
for park in parks:
    url = url_base + park[0]
    driver.get(url)
    print(park_closed())

False
True
True
True


In [7]:
def collect_times2(url_base=url_base, parks=parks):
    wait_list = []
    for park in parks:
        url = url_base + park[0]
        driver.get(url)
        time.sleep(1)
        if park_closed() == False:
            for n in range(2, park[1]):
                if check_ride_down(n) == True:
                    a = ride(n)
                    wait_list+=[[date.today().strftime("%m/%d/%Y"),
                                 datetime.now().strftime("%H:%M"),
                                 park[0],
                                 a[0],
                                 a[1]]]
    return wait_list

In [8]:
def update_file2():
    times = collect_times2()
    if len(times) > 0:
        df_new = pd.DataFrame(np.array(times), columns=['date', 'time', 'park', 'ride', 'wait'])
        df_old = pd.read_csv('wait_time.csv')
        df_combine = pd.concat([df_old, df_new])
        df_combine.to_csv(index=False, path_or_buf='wait_time.csv', mode='w+')

In [9]:
def main2():
    while int(datetime.now().strftime("%H")) < 23:
        if int(datetime.now().strftime("%M")) % 5 == 0:
            update_file2()
            time.sleep(295)

In [ ]:
main2()